In [1]:
######## YCrCb
import cv2
import numpy as np

cap = cv2.VideoCapture(0)
top, right, bottom, left = 10, 350, 225, 590
while(1):

    # Take each frame
    _, frame = cap.read()

    roi = frame[top:bottom, right:left]
    # Convert BGR to HSV
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2YCR_CB)

    # define range of skin color in HSV
    lower_skin = np.array([0,138,67])
    upper_skin = np.array([255,173,133])
    
    # Threshold the HSV image to get only blue colors
    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    

    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(roi,roi, mask= mask)
    
    #blur=cv2.GaussianBlur(mask,(7,7),0)
    
    thresholded = cv2.threshold(mask,
                                50,
                                255,
                                cv2.THRESH_BINARY)[1]
    
    (_, cnts, _) = cv2.findContours(thresholded.copy(),
                                    cv2.RETR_EXTERNAL,
                                    cv2.CHAIN_APPROX_SIMPLE)
    if(len(cnts)!=0):
        segmented = max(cnts, key=cv2.contourArea)
        cv2.drawContours(frame, [segmented + (right, top)], -1, (0, 0, 255)) #red line around hand in ROI of video
    
    cv2.rectangle(frame, (left, top), (right, bottom), (0,255,0), 2)
  
    
    cv2.imshow('frame',frame)
    #cv2.imshow('mask',mask)
    cv2.imshow('res',res)
    
    #cv2.imshow('blur',blur)
    cv2.imshow('thresholded',thresholded)

    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
#press ESC to exit
cv2.destroyAllWindows()